In [85]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Đọc và hợp nhất các tệp CSV
files = glob.glob("D:\\code\\proj_ml\\ML_Project\\data\\saved\\*.csv")
data_list = []

for file in files:
    df = pd.read_csv(file, parse_dates=['Date'])
    data_list.append(df)

# Hợp nhất các DataFrame thành một DataFrame lớn
data = pd.concat(data_list, ignore_index=True)

# Đặt 'Date' làm chỉ mục
data.set_index('Date', inplace=True)

# Sắp xếp dữ liệu theo ngày để đảm bảo thứ tự đúng
data.sort_index(inplace=True)

# Dịch chuyển biến mục tiêu (giá vàng đóng cửa) 1 ngày để dự đoán giá đóng cửa ngày hôm sau
data['next_day_gold_close'] = data['gold_close'].shift(-1)

# Loại bỏ hàng cuối cùng vì không có giá trị mục tiêu
data.drop(data.tail(1).index, inplace=True)

In [86]:
data.dtypes

copper                 float64
crude_oil              float64
DXY                    float64
EURUSD                 float64
gold_open              float64
gold_high              float64
gold_low               float64
gold_close             float64
MSCI                   float64
NASDAQ                 float64
NLR                    float64
platinum               float64
RUT                    float64
silver                 float64
SP500                  float64
VIX                    float64
next_day_gold_close    float64
dtype: object

In [88]:
# Xử lý các giá trị NaN
data = data.dropna()

# Chọn các đặc trưng và biến mục tiêu
features = ['copper', 'crude_oil', 'DXY', 'EURUSD', 'MSCI', 'NASDAQ', 'NLR', 'platinum', 'RUT', 'silver', 'SP500', 'VIX']
target = 'next_day_gold_close'

X = data[features]
y = data[target]

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa các đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Sử dụng GridSearchCV để tìm tham số tốt nhất cho mô hình KNN
param_grid = {'n_neighbors': np.arange(1, 30)}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_scaled, y_train)

# Lấy tham số tốt nhất
best_n_neighbors = grid_search.best_params_['n_neighbors']
print(f"Best number of neighbors: {best_n_neighbors}")

# Tạo và huấn luyện mô hình KNN với tham số tốt nhất
knn = KNeighborsRegressor(n_neighbors=best_n_neighbors)
knn.fit(X_train_scaled, y_train)

# Dự đoán
y_pred = knn.predict(X_test_scaled)


Best number of neighbors: 1


In [93]:
# Chuyển y_test thành mảng NumPy
y_test_array = y_test.values

# Tính toán các chỉ số đánh giá
mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array))
rmse = np.sqrt(mean_squared_error(y_test_array, y_pred))
r2 = r2_score(y_test_array, y_pred)

# In kết quả
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")

Mean Absolute Percentage Error (MAPE): 0.0189
Root Mean Squared Error (RMSE): 40.2728
R2 Score: 0.9840
